In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('master_books.csv')
print(df.shape)
df.head()

(19707, 6)


,title,author,rating,description,language,isbn
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"4.33 avg rating — 6,191,931 ratings",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,0439023483 (ISBN13: 9780439023481)
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,"4.50 avg rating — 2,414,060 ratings",There is a door at the end of a silent corrido...,English,0439358078 (ISBN13: 9780439358071)
2,To Kill a Mockingbird,Harper Lee,"4.28 avg rating — 4,348,656 ratings",The unforgettable novel of a childhood in a sl...,English,NaN
3,Pride and Prejudice,Jane Austen,"4.26 avg rating — 2,886,703 ratings",Alternate cover edition of ISBN 9780679783268S...,English,NaN
4,"Twilight (Twilight, #1)",Stephenie Meyer,"3.59 avg rating — 4,815,701 ratings",About three things I was absolutely positive.\...,English,0316015849 (ISBN13: 9780316015844)


In [3]:
df['language'].value_counts()

English                                15960
Arabic                                   441
Spanish                                  278
French                                   220
German                                   160
Indonesian                               159
Portuguese                               114
Italian                                   90
Dutch                                     86
Turkish                                   77
Persian                                   54
Polish                                    41
Romanian                                  37
Malay                                     33
Czech                                     33
Bulgarian                                 31
Swedish                                   25
Japanese                                  24
Urdu                                      21
Greek, Modern (1453-)                     21
Russian                                   18
Bengali                                   17
Serbian   

In [4]:
df = df[df['language'] == 'English']
df.shape

(15960, 6)

In [5]:
df.isnull().sum()

title             0
author            0
rating            0
description     148
language          0
isbn           2754
dtype: int64

In [6]:
df['title'].value_counts()

The Gift                                   6
The Stand                                  5
The Lost Girls of Rome                     4
Our Endless Numbered Days                  4
Whisper Beach                              4
                                          ..
The Threat Below (Brathius History, #1)    1
Opposite Similarities                      1
All You Zombies                            1
Sentimental Education                      1
The Lying Game (The Lying Game, #1)        1
Name: title, Length: 12365, dtype: int64

In [7]:
seen = set()
def find_doubles(title, author, seen=seen):
    title_author = (title, author)
    if title_author not in seen:
        seen.add(title_author)
        return False
    else:
        return True

In [8]:
df['double'] = df.apply(lambda x: find_doubles(x.title, x.author), axis=1)

In [9]:
df['double'].value_counts()

False    12415
True      3545
Name: double, dtype: int64

In [10]:
mask = ~df['double']
df = df[mask]

In [11]:
df.isnull().sum()

title             0
author            0
rating            0
description     104
language          0
isbn           2025
double            0
dtype: int64

In [12]:
df = df.dropna(subset=['description'])
df.shape

(12311, 7)

In [13]:
import re
text = df.loc[0, 'description'].lower()
re.sub('\W+', ' ', text)

'winning means fame and fortune losing means certain death the hunger games have begun in the ruins of a place once known as north america lies the nation of panem a shining capitol surrounded by twelve outlying districts the capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual hunger games a fight to the death on live tv sixteen year old katniss everdeen regards it as a death sentence when she steps forward to take her sister s place in the games but katniss has been close to dead before and survival for her is second nature without really meaning to she becomes a contender but if she is to win she will have to start making choices that weight survival against humanity and life against love '

In [14]:
df['text'] = [re.sub('\W+', ' ', text.lower()) for text in df['description']]

In [69]:
# find those non-english descriptions with 'english' as language
# join up books from a series

In [71]:
import enchant

In [74]:
english = enchant.Dict("en_US")

def check_english(book, english=english):
    ten_words = book.split()[:10]
    
    word_is_english = []
    
    for word in ten_words:
        word_is_english.append(english.check(word))
    
    english_words = word_is_english.count(True)
    return english_words / len(word_is_english)

In [73]:
df.columns

Index(['title', 'author', 'rating', 'description', 'language', 'isbn',
       'double', 'text'],
      dtype='object')

In [83]:
df['is_english'] = df['text'].apply(check_english)

df.head(10)

,title,author,rating,description,language,isbn,double,text,is_english
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"4.33 avg rating — 6,191,931 ratings",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,0439023483 (ISBN13: 9780439023481),False,winning means fame and fortune losing means ce...,1.0
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,"4.50 avg rating — 2,414,060 ratings",There is a door at the end of a silent corrido...,English,0439358078 (ISBN13: 9780439358071),False,there is a door at the end of a silent corrido...,1.0
2,To Kill a Mockingbird,Harper Lee,"4.28 avg rating — 4,348,656 ratings",The unforgettable novel of a childhood in a sl...,English,NaN,False,the unforgettable novel of a childhood in a sl...,1.0
3,Pride and Prejudice,Jane Austen,"4.26 avg rating — 2,886,703 ratings",Alternate cover edition of ISBN 9780679783268S...,English,NaN,False,alternate cover edition of isbn 9780679783268s...,0.8
4,"Twilight (Twilight, #1)",Stephenie Meyer,"3.59 avg rating — 4,815,701 ratings",About three things I was absolutely positive.\...,English,0316015849 (ISBN13: 9780316015844),False,about three things i was absolutely positive f...,0.9
5,The Book Thief,Markus Zusak (Goodreads Author),"4.37 avg rating — 1,767,200 ratings",It is 1939. Nazi Germany. The country is holdi...,English,0375831002 (ISBN13: 9780375831003),False,it is 1939 nazi germany the country is holding...,0.8
6,The Chronicles of Narnia (Chronicles of Narnia...,C.S. Lewis,"4.26 avg rating — 498,771 ratings","Journeys to the end of the world, fantastic cr...",English,NaN,False,journeys to the end of the world fantastic cre...,1.0
7,Animal Farm,George Orwell,"3.94 avg rating — 2,631,549 ratings",Librarian's note: There is an Alternate Cover ...,English,0451526341 (ISBN13: 9780451526342),False,librarian s note there is an alternate cover e...,1.0
8,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,"4.60 avg rating — 107,831 ratings","This four-volume, boxed set contains J.R.R. To...",English,0345538374 (ISBN13: 9780345538376),False,this four volume boxed set contains j r r tolk...,0.9
9,Gone with the Wind,Margaret Mitchell,"4.30 avg rating — 1,054,418 ratings","Scarlett O'Hara, the beautiful, spoiled daught...",English,0446675539 (ISBN13: 9780446675536),False,scarlett o hara the beautiful spoiled daughter...,0.8


In [84]:
df['is_english'].value_counts()

1.000000    5198
0.900000    3682
0.800000    2494
0.700000     674
0.600000     154
0.500000      45
0.300000      14
0.200000      12
0.400000      11
0.100000       9
0.857143       4
0.750000       4
0.888889       3
0.000000       3
0.285714       1
0.875000       1
0.714286       1
0.444444       1
Name: is_english, dtype: int64

In [91]:
df.query("is_english == 0.5")

,title,author,rating,description,language,isbn,double,text,is_english
18,Jane Eyre,Charlotte Brontë,"4.12 avg rating — 1,577,046 ratings","Clicca qui per l'edizione ""Storie senza tempo""...",English,0142437204 (ISBN13: 9780142437209),False,clicca qui per l edizione storie senza tempo o...,0.5
619,"The Goose Girl (The Books of Bayern, #1)",Shannon Hale (Goodreads Author),"4.13 avg rating — 126,672 ratings","Anidora-Kiladra Talianna Isilee, Crown Princes...",English,1582349908 (ISBN13: 9781582349909),False,anidora kiladra talianna isilee crown princess...,0.5
963,"Wizard and Glass (The Dark Tower, #4)",Stephen King (Goodreads Author),"4.25 avg rating — 158,404 ratings","Roland, Eddie, Susannah, Jake, and Jake’s pet ...",English,0340829788 (ISBN13: 9780340829783),False,roland eddie susannah jake and jake s pet bumb...,0.5
2058,Team of Rivals: The Political Genius of Abraha...,Doris Kearns Goodwin,"4.29 avg rating — 145,471 ratings",Winner of the Lincoln PrizeAcclaimed historian...,English,0743270754 (ISBN13: 9780743270755),False,winner of the lincoln prizeacclaimed historian...,0.5
2952,"The Code of the Woosters (Jeeves, #7)",P.G. Wodehouse,"4.35 avg rating — 21,265 ratings","Take Gussie Fink-Nottle, Madeline Bassett, old...",English,0393339815 (ISBN13: 9780393339819),False,take gussie fink nottle madeline bassett old p...,0.5
3107,"Broken Point (The Wayward Gifted, #1)",Donna K. Childree (Goodreads Author),3.87 avg rating — 75 ratings,Samantha Leigh and Steuart James DuBoise resid...,English,1483947823 (ISBN13: 9781483947822),False,samantha leigh and steuart james duboise resid...,0.5
3913,The Company,Robert Littell,"4.23 avg rating — 4,729 ratings",Robert Littell does for the CIA what Mario Puz...,English,0330372890 (ISBN13: 9780330372893),False,robert littell does for the cia what mario puz...,0.5
4184,The Hero's Guide to Saving Your Kingdom (The L...,Christopher Healy (Goodreads Author),"4.05 avg rating — 10,712 ratings",Prince Liam. Prince Frederic. Prince Duncan. P...,English,0062117432 (ISBN13: 9780062117434),False,prince liam prince frederic prince duncan prin...,0.5
4540,"Separation of Power (Mitch Rapp, #5)",Vince Flynn,"4.31 avg rating — 40,949 ratings",\r\n CIA superagent Mitch Rapp is back in act...,English,0671047345 (ISBN13: 9780671047344),False,cia superagent mitch rapp is back in action i...,0.5
5348,The Iron-Jawed Boy and the Hand of the Moon (G...,Nikolas Lee,4.46 avg rating — 133 ratings,My Official Youtube: https://www.youtube.com/w...,English,1494318326 (ISBN13: 9781494318321),False,my official youtube https www youtube com watc...,0.5


In [76]:
df.iloc[415]

title                                          Faust, First Part
author                                Johann Wolfgang von Goethe
rating                          3.93 avg rating — 57,076 ratings
description    Wielki uczony - wciąż spragniony wiedzy o sens...
language                                                 English
isbn                          0553213482 (ISBN13: 9780553213485)
double                                                     False
text           wielki uczony wciąż spragniony wiedzy o sensie...
is_english                                                   0.1
Name: 483, dtype: object

## First run
    Just using TfidfVect for vectorization
    using default english stop words and no min/max df stuff

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
vect = TfidfVectorizer(stop_words='english')
data = [''.join(x) for x in df['text']]
data

['winning means fame and fortune losing means certain death the hunger games have begun in the ruins of a place once known as north america lies the nation of panem a shining capitol surrounded by twelve outlying districts the capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual hunger games a fight to the death on live tv sixteen year old katniss everdeen regards it as a death sentence when she steps forward to take her sister s place in the games but katniss has been close to dead before and survival for her is second nature without really meaning to she becomes a contender but if she is to win she will have to start making choices that weight survival against humanity and life against love ',
 'there is a door at the end of a silent corridor and it s haunting harry pottter s dreams why else would he be waking in the middle of the night screaming in terror harr

In [20]:
vect.fit(data)
sparse = vect.transform(data)

dtm = pd.DataFrame(sparse.todense(), columns=vect.get_feature_names())

dtm.head()

,00,000,001,007,01,0140422536,0141183829,02,020,03,...,धर,मपद,美しさと哀しみと,ﬁerce,ﬁnd,ﬁred,ﬁrst,ﬂame,ﬂavorful,ﬂavors
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from sklearn.neighbors import NearestNeighbors

In [22]:
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

In [23]:
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [25]:
new = ['A magical boy fights evil with his friends at school']
new_vected = vect.transform(new)
new_df = pd.DataFrame(new_vected.todense(), columns=vect.get_feature_names())
nn.kneighbors(new_df)

(array([[1.20973465, 1.25043285, 1.27715505, 1.28113096, 1.28365332,
         1.2915138 , 1.30107389, 1.30129973, 1.30196266, 1.30539302]]),
 array([[ 2581,  5852,  4876,  8444, 11076, 11764,  7608,  4082,  4377,
         10906]], dtype=int64))

In [26]:
df.iloc[2581]

title                                    That Boy (That Boy, #1)
author                           Jillian Dodd (Goodreads Author)
rating                          3.93 avg rating — 33,897 ratings
description    Fall in love with the boy next door in this sw...
language                                                 English
isbn                                                         NaN
double                                                     False
text           fall in love with the boy next door in this sw...
Name: 2778, dtype: object

In [27]:
df.iloc[5852]

title                                 The Prophecy of the Stones
author                                              Flavia Bujor
rating                           3.50 avg rating — 5,768 ratings
description    In a magical realm, three teenage girls--Jade,...
language                                                 English
isbn                          0786856556 (ISBN13: 9780786856558)
double                                                     False
text           in a magical realm three teenage girls jade op...
Name: 6483, dtype: object

In [29]:
df.iloc[8444]

title          Sweet Remembrance: The Little Match Girl Retel...
author                       Emily Ann Putzke (Goodreads Author)
rating                              4.24 avg rating — 21 ratings
description    In the despair of the Warsaw Ghetto, a young J...
language                                                 English
isbn                                                         NaN
double                                                     False
text           in the despair of the warsaw ghetto a young je...
Name: 10116, dtype: object

## End first run

## Second run
        use min/max df and ngram_range hyperparameters
        somehow doing significantly worse, uncovered non-english books sneaking through

In [57]:
vect = TfidfVectorizer(stop_words='english', min_df=0.025, max_df=0.975)
data = [''.join(x) for x in df['text']]
data
vect.fit(data)
sparse = vect.transform(data)

dtm = pd.DataFrame(sparse.todense(), columns=vect.get_feature_names())

dtm.head()

,able,acclaimed,action,adventure,age,ago,alive,america,american,ancient,...,works,world,writer,writing,written,wrong,year,years,york,young
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.184122,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.132832,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.175865,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.173685,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.134342
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [58]:
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [59]:
new = ['A magical boy fights evil with his friends at school']
new_vected = vect.transform(new)
new_df = pd.DataFrame(new_vected.todense(), columns=vect.get_feature_names())
nn.kneighbors(new_df)

(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 array([[ 415, 3940, 2847, 2620, 1657, 1179, 2562,  395, 1400, 3537]],
       dtype=int64))

In [68]:
df.iloc[2562]

title            Chaos Walking: A Trilogy (Chaos Walking #1-3)
author                                            Patrick Ness
rating                         4.49 avg rating — 3,727 ratings
description    3 titles in one from the Chaos Walking Trilogy.
language                                               English
isbn                        1406334332 (ISBN13: 9781406334333)
double                                                   False
text           3 titles in one from the chaos walking trilogy 
Name: 2757, dtype: object

## End Second run

In [99]:
import spacy

In [100]:
nlp = spacy.load('en_core_web_sm')

In [101]:
text = df.loc[:5, 'description']
text = ''.join(text).replace('\r', '').replace('\n', '')
text

'WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister\'s place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming 

In [102]:
from spacy.tokenizer import Tokenizer

tokenizer = Tokenizer(nlp.vocab)

In [103]:
def make_tokens(text, tokenizer=tokenizer):
    return [token.text for token in tokenizer(text) if token.is_stop == False] 

In [104]:
df['tokens'] = df['text'].apply(make_tokens)
df['tokens']

0        [winning, means, fame, fortune, losing, means,...
1        [door, end, silent, corridor, s, haunting, har...
2        [unforgettable, novel, childhood, sleepy, sout...
3        [alternate, cover, edition, isbn, 978067978326...
4        [things, absolutely, positive, edward, vampire...
                               ...                        
15589    [tradition, bury, heart, wounded, knee, stunni...
15590    [ , woman, year, history, karyukai, come, forw...
15591    [true, story, individual, s, struggle, self, i...
15592    [jack, reacher, hero, loner, soldier, soldier,...
15593    [won, fame, freedom, gory, pits, rome, s, colo...
Name: tokens, Length: 12311, dtype: object

In [105]:
%%timeit

tokens = []

for doc in tokenizer.pipe(df['text'], batch_size=100):
    
    doc_tokens = [token.text for token in doc if token.is_stop == False]
    
    tokens.append(doc_tokens)

KeyboardInterrupt: 

In [54]:
%%timeit
df['tokens'] = df['text'].apply(make_tokens)

6.71 s ± 249 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
df['tokens']

0        [WINNING, MEANS, FAME, FORTUNE, LOSING, MEANS,...
1        [door, end, silent, corridor, s, haunting, Har...
2        [unforgettable, novel, childhood, sleepy, Sout...
3        [Alternate, cover, edition, ISBN, 978067978326...
4        [things, absolutely, positive, Edward, vampire...
                               ...                        
15589    [tradition, Bury, Heart, Wounded, Knee, stunni...
15590    [ , woman, year, history, karyukai, come, forw...
15591    [true, story, individual, s, struggle, self, i...
15592    [Jack, Reacher, Hero, Loner, Soldier, Soldier,...
15593    [won, fame, freedom, gory, pits, Rome, s, Colo...
Name: tokens, Length: 12311, dtype: object

In [ ]:
df['lemmas'] = [token.lemma_ for token in ]

In [19]:
[token.lemma_ for token in doc if token.pos_ == 'VERB']

['mean',
 'mean',
 'have',
 'know',
 'lie',
 'shine',
 'surround',
 'keep',
 'force',
 'send',
 'participate',
 'regard',
 'step',
 'take',
 'mean',
 'become',
 'win',
 'will',
 'start',
 'make',
 'weight',
 '’',
 'would',
 'wake',
 'scream',
 'poison',
 'loom',
 'pale',
 'grow',
 'Must',
 'name',
 'can',
 'stop',
 'must',
 'discover',
 'depend',
 'rock',
 'kill',
 'become',
 'publish',
 'go',
 'win',
 'make',
 'win',
 'move',
 'kill',
 'take',
 'translate',
 'claim',
 'consider',
 'regard',
 'remain',
 'call',
 'appear',
 'sparkle',
 'dance',
 'make',
 'know',
 'may',
 'thirst',
 'hold',
 'will',
 'change',
 'pick',
 'hide',
 'leave',
 'begin',
 'play',
 'learn',
 'read',
 'steal',
 'find',
 'hide',
 'open',
 'close',
 'craft',
 'write',
 'burn',
 'win',
 'give',
 'publish']

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [107]:
vect = TfidfVectorizer()

data = [' '.join(x) for x in df['tokens']]

vect.fit(data)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [108]:
sparse = vect.transform(data)

dtm = pd.DataFrame(sparse.todense(), columns=vect.get_feature_names())

dtm.head()

,00,000,001,007,01,0140422536,0141183829,02,020,03,...,धर,मपद,美しさと哀しみと,ﬁerce,ﬁnd,ﬁred,ﬁrst,ﬂame,ﬂavorful,ﬂavors
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
from sklearn.neighbors import NearestNeighbors

In [110]:
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

In [111]:
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [118]:
new = 'A magical boy fights evil with his friends at school'
new_doc = make_tokens(new)
new_vected = vect.transform(new_doc)
new_df = pd.DataFrame(new_vected.todense(), columns=vect.get_feature_names())
nn.kneighbors(new_df)

(array([[1.20233787, 1.22798083, 1.23712537, 1.24971542, 1.25385554,
         1.25970846, 1.26429497, 1.27240709, 1.27832788, 1.27933477],
        [0.78949975, 1.03335467, 1.06692127, 1.08155511, 1.10601946,
         1.17804631, 1.18449841, 1.1847156 , 1.19782272, 1.19915932],
        [1.16847743, 1.23869497, 1.27533981, 1.27732557, 1.28939594,
         1.29081054, 1.29116769, 1.29310696, 1.29692682, 1.30315604],
        [1.17764911, 1.20934183, 1.21000525, 1.23982188, 1.25121311,
         1.25782809, 1.26577317, 1.26728182, 1.27495831, 1.27686157],
        [1.13643085, 1.15978976, 1.17308088, 1.17628656, 1.20876113,
         1.22995901, 1.23851182, 1.24840642, 1.25004244, 1.25269511],
        [1.1530487 , 1.17924903, 1.20931369, 1.21031704, 1.22022148,
         1.22664071, 1.22723947, 1.22883821, 1.23225016, 1.23815005]]),
 array([[ 5852,  6706, 10464,  6658,  2835, 10275,  5234,   436,  7925,
          5301],
        [ 2581, 11764,  7608, 11339,  2941,  4226,  6955,  8388, 11705,
   

In [119]:
df.iloc[8444]

title          Sweet Remembrance: The Little Match Girl Retel...
author                       Emily Ann Putzke (Goodreads Author)
rating                              4.24 avg rating — 21 ratings
description    In the despair of the Warsaw Ghetto, a young J...
language                                                 English
isbn                                                         NaN
double                                                     False
text           in the despair of the warsaw ghetto a young je...
tokens         [despair, warsaw, ghetto, young, jewish, woman...
Name: 10116, dtype: object

In [114]:
df.iloc[6706]

title           Circles in the Stream (Avalon: Web of Magic, #1)
author                         Rachel Roberts (Goodreads Author)
rating                           4.09 avg rating — 2,912 ratings
description    It's every girl's fantasy to find herself pull...
language                                                 English
isbn                          1933164646 (ISBN13: 9781933164649)
double                                                     False
text           it s every girl s fantasy to find herself pull...
tokens         [s, girl, s, fantasy, find, pulled, new, magic...
Name: 7531, dtype: object